# A demonstration of a Little Sheens Jupyter notebook

This notebook uses a simple fork of Little Sheens that has a tool that generates an `npm` module based on the existing Javascript sources. The actual kernel that's running is [ijavascript](https://github.com/n-riesco/ijavascript), which was easy enough to install.

So now there's literate Sheens programming?

In [24]:
var s = require('littlesheens')

Let's do a quick test. The module exposes the `match` function:

In [2]:
s.match(null, {"x":"?x"}, {"x":1}, {})

[ { '?x': 1 } ]

Now let's define a spec (and write it in JSON). This example is straight from the [Little Sheens](https://github.com/Comcast/littlesheens) repo.

In [3]:
var spec = {
  "doc": "A machine that double numbers and protests requests for doubling non-numbers.",
  "name": "double",
  "nodes": {
    "cleanup": {
      "action": {
        "interpreter": "ecmascript",
        "source": "delete _.bindings[\"?x\"];\nreturn _.bindings;"
      },
      "branching": {
        "branches": [
          {
            "target": "listen"
          }
        ]
      }
    },
    "complain": {
      "action": {
        "interpreter": "ecmascript",
        "source": "_.out({bad: _.bindings[\"?x\"]});\nreturn _.bindings;"
      },
      "branching": {
        "branches": [
          {
            "target": "cleanup"
          }
        ]
      }
    },
    "double": {
      "action": {
        "interpreter": "ecmascript",
        "source": "_.out({doubled: _.bindings[\"?x\"]*2});\n_.bindings.count++;\nreturn _.bindings;"
      },
      "branching": {
        "branches": [
          {
            "target": "cleanup"
          }
        ]
      }
    },
    "listen": {
      "branching": {
        "branches": [
          {
            "guard": {
              "interpreter": "ecmascript",
              "source": "var bs = _.bindings;\nvar f = parseFloat(bs[\"?x\"]);\nif (isNaN(f)) {\n   return nil;\n}\nbs[\"?x\"] = f;\nreturn bs;"
            },
            "pattern": "{\"double\":\"?x\"}\n",
            "target": "double"
          },
          {
            "pattern": "{\"double\": \"?x\"}\n",
            "target": "complain"
          }
        ],
        "type": "message"
      }
    },
    "start": {
      "branching": {
        "branches": [
          {
            "target": "listen"
          }
        ]
      }
    }
  },
  "parsepatterns": true
};

Can we actually use that spec?

In [4]:
var walked = s.walk(null, spec, null, {"double":2})

In [5]:
console.log("I actually got output: " + JSON.stringify(walked.emitted))

I actually got output: [{"doubled":4}]


Sweet victory is ours.

Now let's check out the profiling support we have.

In [6]:
s.times.enable()

In [7]:
s.walk(null, spec, null, {"double":2})

{ to: { node: 'listen', bs: { count: null } },
  consumed: true,
  emitted: [ { doubled: 4 } ] }

In [8]:
s.times.summary()

{ step: { ms: 39, n: 5 },
  match: { ms: 0, n: 1 },
  sandbox: { ms: 38, n: 3 } }

In [9]:
for (var i = 0; i < 10; i++) { s.walk(null, spec, null, {"double":2}); }

{ to: { node: 'listen', bs: { count: null } },
  consumed: true,
  emitted: [ { doubled: 4 } ] }

In [10]:
s.times.summary()

{ step: { ms: 368, n: 55 },
  match: { ms: 1, n: 11 },
  sandbox: { ms: 366, n: 33 } }

In [13]:
var sandboxes = s.times.summary().sandbox;
sandboxes.ms/sandboxes.n;

11.090909090909092

About 11ms on average for a sandboxed action execution. Okay, I'm running in a small VM on an older Macbook.

In [15]:
var emitted = [];
for (var i = 0; i < 10; i++) {
    var walked = s.walk(null, spec, null, {"double":i});
    emitted = emitted.concat(walked.emitted);
}
emitted;

[ { doubled: 0 },
  { doubled: 2 },
  { doubled: 4 },
  { doubled: 6 },
  { doubled: 8 },
  { doubled: 10 },
  { doubled: 12 },
  { doubled: 14 },
  { doubled: 16 },
  { doubled: 18 } ]

In [16]:
s.times.summary()

{ step: { ms: 1295, n: 155 },
  match: { ms: 2, n: 31 },
  sandbox: { ms: 1289, n: 93 } }

In [21]:
function go(n) {
    s.times.reset();
    var emitted = [];
    for (var i = 0; i < 10; i++) {
        var walked = s.walk(null, spec, null, {"double":i});
        emitted = emitted.concat(walked.emitted);
    }
    var times = s.times.summary();
    return {meanSandbox: times.sandbox.ms/times.sandbox.n, walked: n};
}

In [23]:
go(100)

{ meanSandbox: 13.933333333333334, walked: 100 }